In [1]:
import requests
import json
from datetime import datetime, timedelta, timezone
import pandas as pd

## Get historical air quality data

In [4]:
# Get air quality data for Leuven
# Period: Jan 1, 2022 to Dec 31, 2022
api_key = "ec722f11f234fb9a316e2580e6e2019e"
lat = 50.88
lon= 4.7
start_date = "1640995200"
end_date = "1672531200"

url = f"http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start_date}&end={end_date}&appid={api_key}"
# url = f"http://api.openweathermap.org/data/2.5/air_pollution?lat={lat}&lon={lon}&appid={api_key}"
#url = f"http://api.openweathermap.org/data/2.5/air_pollution/forecast?lat={lat}&lon={lon}&appid={api_key}"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Error:", response.status_code)

In [6]:
# Historical
df = pd.DataFrame(data['list'])
df["dt"] = pd.to_datetime(df["dt"], unit="s")
components_df = pd.json_normalize(df["components"])
aqi_df = pd.json_normalize(df['main'])
df = pd.concat([df, components_df, aqi_df], axis=1)
df.drop(["components", "main"], axis=1, inplace=True)
df.to_csv("air_quality_full.csv")
df.head(10)

,dt,co,no,no2,o3,so2,pm2_5,pm10,nh3,aqi
0,2022-01-01 00:00:00,253.68,0.00,8.83,39.34,1.82,3.68,5.20,0.79,1
1,2022-01-01 01:00:00,250.34,0.00,8.05,38.62,1.86,3.69,5.27,0.79,1
2,2022-01-01 02:00:00,247.00,0.00,7.11,38.27,1.71,3.44,4.99,0.78,1
3,2022-01-01 03:00:00,243.66,0.00,6.17,38.62,1.33,2.93,4.26,0.72,1
4,2022-01-01 04:00:00,257.02,0.00,6.85,37.19,1.03,3.02,4.07,0.68,1
5,2022-01-01 05:00:00,290.39,0.00,10.11,32.54,0.95,4.49,5.46,0.69,1
6,2022-01-01 06:00:00,330.45,0.02,15.42,26.82,1.18,6.74,7.79,0.73,1
7,2022-01-01 07:00:00,337.12,0.04,18.68,23.96,1.48,7.13,8.26,0.76,1
8,2022-01-01 08:00:00,307.08,0.04,18.16,25.03,1.64,5.74,6.81,0.71,1
9,2022-01-01 09:00:00,273.70,0.09,15.42,28.61,1.59,4.23,5.20,0.66,1


In [2]:
import pandas as pd
df = pd.read_csv('tripadvisor_leuven.csv')

## Get historical traffic data

In [27]:
url = "https://telraam-api.net/v1/reports/traffic"
headers = {
    'X-Api-Key': 'Z9Qoyuy9Yf99nf2I0Myig4t0ftuUmY81ahBIVZH4'
}

start_date = pd.to_datetime('2022-01-01')
end_date = pd.to_datetime('2023-01-01')

# define the duration for each loop (3 months)
loop_duration = pd.DateOffset(months=3)

# define the list of sensor IDs
counter_ids = ['9000000627', '347690', '9000000674', '9000000773', '347931', '347860', 
               '9000000764', '9000000672', '347948', '9000001547', '347365', '349054', '9000000681']

full_data = pd.DataFrame()

# loop through the counter IDs
for counter_id in counter_ids:
    # retrieve data for each 3-month period
    current_date = start_date
    while current_date < end_date:
        loop_start_date = current_date
        loop_end_date = loop_start_date + loop_duration

        body = {
            "id": counter_id,
            "time_start": loop_start_date.strftime("%Y-%m-%d %H:%M:%SZ"),
            "time_end": loop_end_date.strftime("%Y-%m-%d %H:%M:%SZ"),
            "level": "segments",
            "format": "per-hour"
        }
        payload = str(body)

        response = requests.post(url, headers=headers, data=payload)
        json_data = response.json()
        loop_data = pd.DataFrame(json_data['report'])
        full_data = pd.concat([full_data, loop_data])

        # update the current date for the next loop
        current_date += loop_duration



In [24]:
location = {
    '9000000627': 'redingenhof',
    '347690': 'Kapucijnenvoer',
    '9000000674': 'Tiensestraat',
    '9000000773': 'Bondgenotenlaan',
    '347931': 'Vital Decostersstraat',
    '347860': 'BROUWERSSTRAAT',
    '9000000764': 'Fonteinstraat 137 b 301',
    '9000000672':'Petermannenstraat',
    '347948' :'Ridderstraat',
    '9000001547': 'Jan Pieter Minckelersstraat',
    '347365': 'Dekenstraat',
    '349054': 'Pleinstraat',
    '9000000681':'Bierbeekstraat'
}

In [40]:
full_data['location'] = full_data.segment_id.astype('str').map(location)
full_data.to_csv('traffic_leuven_2022.csv', index=False)

In [44]:
full_data

,instance_id,segment_id,date,interval,uptime,heavy,car,bike,pedestrian,heavy_lft,...,bike_lft,bike_rgt,pedestrian_lft,pedestrian_rgt,direction,car_speed_hist_0to70plus,car_speed_hist_0to120plus,timezone,v85,location
0,-1,9000000627,2022-01-01T00:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,redingenhof
1,-1,9000000627,2022-01-01T01:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,redingenhof
2,-1,9000000627,2022-01-01T02:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,redingenhof
3,-1,9000000627,2022-01-01T03:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,redingenhof
4,-1,9000000627,2022-01-01T04:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,redingenhof
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2201,-1,9000000681,2022-12-31T19:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,Bierbeekstraat
2202,-1,9000000681,2022-12-31T20:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,Bierbeekstraat
2203,-1,9000000681,2022-12-31T21:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,Bierbeekstraat
2204,-1,9000000681,2022-12-31T22:00:00.000Z,hourly,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Europe/Brussels,NaN,Bierbeekstraat
